# GlobalAveragePooling (GAP)
- Feature map의 채널별로 평균값을 추출 1 x 1 x channel 의 Feature map을 생성
- `model.add(keras.layers.GlobalAveragePooling2D())`
![image-2.png](attachment:image-2.png)

- Feature Extraction layer에서 추출한 Feature map을 Classifier layer로 Flatten해서 전달하면 많은 연결노드와 파라미터가 필요하게된다. GAP를 사용하면 노드와 파라미터의 개수를 효과적으로 줄일 수 있다.
- Feature map의 채널수가 많을 경우 GAP를 사용하는 것이 효과적이나 채널수가 적다면 Flatten을 사용하는 것이 좋다.
![image-2.png](attachment:image-2.png)

In [1]:
# 이미지 다운로드
import gdown
url = 'https://drive.google.com/uc?id=1nBE3N2cXQGwD8JaD0JZ2LmFD-n3D5hVU'
fname = 'cats_and_dogs_small.zip'
gdown.download(url, fname, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1nBE3N2cXQGwD8JaD0JZ2LmFD-n3D5hVU
To: /content/cats_and_dogs_small.zip
90.8MB [00:00, 105MB/s] 


'cats_and_dogs_small.zip'

In [2]:
!mkdir data

In [3]:
## 압축 풀기
!unzip -q ./cats_and_dogs_small.zip -d data/cats_and_dogs_small

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def get_generators():
    '''
    train, validation, test generator를 생성해서 반환.
    train generator는 image 변환 처리
    '''
    train_dir = './data/cats_and_dogs_small/train'
    validation_dir = './data/cats_and_dogs_small/validation'
    test_dir = './data/cats_and_dogs_small/test'
    train_datagen = ImageDataGenerator(rescale=1/255,
                                       rotation_range=40,
                                       brightness_range=(0.7,1.3),
                                       zoom_range=0.2,
                                       horizontal_flip=True)
    test_datagen = ImageDataGenerator(rescale=1/255) #validation/test에서 사용
    # generator 들 생성
    train_generator = train_datagen.flow_from_directory(train_dir,
                                                        target_size=(150,150),
                                                        batch_size=N_BATCHS,
                                                        class_mode='binary')
    val_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150,150),
                                                        batch_size=N_BATCHS,
                                                        class_mode='binary')
    test_generator = test_datagen.flow_from_directory(test_dir,
                                                        target_size=(150,150),
                                                        batch_size=N_BATCHS,
                                                        class_mode='binary')
    return train_generator, val_generator, test_generator

![image.png](attachment:image.png)

## Transfer learning (전이학습)
- 큰 데이터 셋을 이용해 미리 학습된 pre-trained Model의 Weight를 사용하여 현재 하려는 예측 문제에 활용. 
- ### Convolution base(Feature Extraction 부분)만 활용
    - Convolution base는 이미지에 나타나는 일반적인 특성을 파악하기 위한 부분이므로 재사용할 수 있다.
    - Classifier 부분은 학습하려는 데이터셋의 class들에 맞게 변경 해야 하므로 재사용할 수 없다.
- Pretrained Convlution layer의 활용 
    - Feature extraction
        - 학습시 학습되지 않고 Feature를 추출하는 역할만 한다.
    - Fine tuning
        - 학습시 Pretrained Covolution layer도 같이 학습해서 내 데이터셋에 맞춘다.

## Feature extraction
- 기존의 학습된 network에서 fully connected layer를 제외한 나머지 weight를 고정하고 새로운 목적에 맞는 fully connected layer를 추가하여 추가된 weight만 학습하는 방법
- `tensorflow.keras.applications` module이 지원하는  image classification models
    - (https://www.tensorflow.org/api_docs/python/tf/keras/applications)    
![image.png](attachment:image.png)





> ### ImageNet
>    - 웹상에서 수집한 약 1500만장의 라벨링된 고해상도 이미지로 약 22,000개 카테고리로 구성된다.

> ### ILSVRC(ImageNet Large Scale Visual Recognition Challenge) 대회
>   - 2010년 부터 2017년 까지 진행된 컴퓨터 비전 경진대회.
>   - ImageNet의 이미지중 **1000개 카테고리 약 120만장의 학습용이미지, 5만장의 검증 이미지, 15만장의 테스트 이미지를** 이용해 대회를 진행한다.
>   - **2012년** CNN기반 딥러닝 알고리즘인 **AlexNet**이 2위와 큰 차이로 우승하며 이후 딥러닝 알고리즘이 대세가 되었다. 특히 2015년 우승한 ResNet은 0.036의 에러율을 보이며 우승했는데 이는 사람이 에러율이라 알려진 0.05 보다 높은 정확도였다.
>   - ILSVRC에서 우승하거나 좋은 성적을 올린 모델들이 컴퓨터 비전분야 발전에 큰 역할을 해왔으며 이후 다양한 딥러닝 모델의 백본(backbone)으로 사용되고 있다.

![image.png](attachment:image.png)

##  VGG16 모델
- ImageNet ILSVRC Challenge 2014에서 2등한 모델로 Simonyan and Zisserman(Oxford Univ.)에 의해 제안
    - VGGNet이 준우승을 하긴 했지만, 구조의 간결함과 사용의 편이성으로 인해 1등한 GoogLeNet보다 더 각광받았다
- 단순한 구조로 지금까지 많이 사용.
- 총 16개 layer로 구성됨.
- 네트워크 깊이가 어떤 영향을 주는 지 연구 하기 위해 설계된 네트워크로 동일한 kernel size에 convolution의 개수를 늘리는 방식으로 구성됨.
    - 11 layer, 13 layer, 16 layer, 19 layer 의 네트워크를 테스트함. 
    - 19 layer의 성능이 16 layer보다 크게 나아지지 않음
- Filter의 수가 64, 128, 256, 512 두 배씩 커짐 
- 항상 $3 \times 3$ filter, Stride=1, same padding, $2\times 2$ MaxPooling 사용
    - 이전 AlexNet이 5 X 5 필터를 사용했는데 VGG16은 3 X 3 필터 두개를 쌓아 사용했다.
        - 3 x 3 필터 두개를 쌓는 것이 5 x 5  하나는 사용하는 보다 더 적은 파라미터를 사용하며 성능이 더 좋았다.
    - Feature map의 사이즈를 convolution layer가 아닌 Max Pooling 을 사용해 줄여줌.
- VGG16의 단점은 마지막에 분류를 위해 Fully Connected Layer 3개를 붙여 파라미터 수가 너무 많아 졌다. 약 1억4천만 개의 parameter(가중치)중 1억 2천만개 정도가 Fully Connected Layer의 파라미터 임.
![image-3.png](attachment:image-3.png)

## ResNet (Residual Networks)
- 이전 모델들과 비교해 shortcut connection기법을 이용해 Layer수를 획기적으로 늘린 CNN 모델로 ILSVRC 2015년 대회에서 우승을 차지함.

![image.png](attachment:image.png)

- 레이어를 깊게 쌓으면 성능이 더 좋아 지지 않을까? 실제는 Test 셋 뿐만 아니라 Train Set에서도 성능이 나쁘게 나옴.
- Train set에서도 성능이 나쁘게 나온 것은 최적화 문제로 보고, 레이어를 깊게 쌓으면 최적화 하기가 어렵다고 생각함. 
![image-2.png](attachment:image-2.png)

### Idea
![image-3.png](attachment:image-3.png)

- 입력값을 그대로 출력하는 identity block 을 사용하면 성능이 떨어지지는 않는다.
- 그럼 Convloution block을 identity block으로 만들면 최소한 성능은 떨어지지 않고 깊은 Layer를 쌓을 수 있지 않을까?

### Solution
- Residual block
![image.png](attachment:image.png)

![image.png](attachment:image.png)

- 기존 Layer들의 목표는 입력값인 X를 출력값인 Y로 최적의 매핑할 수 있는 함수 H(X)를 찾는 것이다. 그래서 H(X) – Y 가 최소값이 되는 방향으로 학습을 진행하면서 H(X)를 찾음. 그런데 레이어가 깊어지면서 최적화에 어려움으로 성능이 떨어지는 문제가 발생

- ResNet은 layer를 통과해서 나온 값이 **입력값과 동일하게 만드는 것을 목표로 하는 Identity block을** 구성한다.
- Identity block은 입력값 X를 레이어를 통과시켜서 나온 Y에 입력값 X를 더해서 합치도록 구성한다.

$$\large H(x) = F(x) + x\\x: input,\;H(x): output,\;F(x): layer통과값$$ 
 
 
- 목표는 $H(x)$(레이어통과한 값) 가 input인 x와 동일한 것이므로 F(x)를 0으로 만들기 위해 학습을 한다. 
- $F(x)$는 **잔차(Residual)**가 된다. 그리고 잔차인 $F(x)$가 0이 되도록 학습하는 방식이므로 Residual Learning이라고 한다.
- 입력인 x를 직접 전달하는 것을 **shortcut connection** 또는 **identity mapping** 또는 **skip connection** 이라고 한다.
    - 이 shortcut은 파라미터 없이 단순히 값을 더하는 구조이므로 연산량에 크게 영향이 없다.
- 그리고 Residual을 찾는 레이어를 **Residual Block, Identity Block** 이라고 한다.      

### 성능향상
- $H(x) = F(x) + x$ 을 $x$에 대해 미분하면 최소한 1이므로 Gradient Vanishing 문제를 극복한다.
- 잔차학습이라고 하지만 Residual block 은 Convolution Layer와 Activation Layer로 구성되어 있기 때문에 이 Layer를 통과한 Input으로 부터 Feature map을 추출하는 과정은 진행되며 레이어가 깊으므로 다양한 더욱 풍부한 특성들을 추출하게 되어 성능이 향상된다.
  

### ResNet 구조
- Residual block들을 쌓는 구조
    - 일반 Convolution Layer(backbone)을 먼저 쌓고 Identity(Residual) block들을 계속 쌓는다.
- 모든 Identity block은 두개의 3X3 conv layer로 구성됨.
- 일정 레이어 수별로 filter의 개수를 두배로 증가시키며 stride를 2로 하여 downsampling 함. (Pooling Layer는 Identity block의 시작과 마지막에만 적용)
![image.png](attachment:image.png)

![image.png](attachment:image.png)

## Pretrained Model 사용
- tensorflow.keras.applications 패키지를 통해 제공
- 모델이름이 클래스이름
    - VGG16, ResNet153 등등
- 생성자 매개변수
    - `weights`: 모형의 학습된 weight. 기본값- 'imagenet'
    - `include_top`: fully connected layer를 포함할지 여부. True 포함시킴, False: 포함 안 시킴
    - `input_shape`: 사용자가 입력할 이미지의 크기 shape. 3D 텐서로 지정. (높이, 너비, 채널). 기본값: (224,224, 3)

In [6]:
from tensorflow.keras.applications import VGG16, ResNet50V2

# conv_base = VGG16(weights='imagenet',  # imagenet 데이터셋을 학습한 가중치(파라미터) 사용 - imagenet(default)
#                   include_top=False,  # Classfication(Fully Connected Layer)는 가져오지 않겠다.(False) -> 보통 False로 지정
#                   input_shape=(150, 150, 3),
#                   )
conv_base = ResNet50V2(weights='imagenet',
                       include_top=False,
                       input_shape=(150, 150, 3),
                       )

94674944/94668760 [==============================] - 1s 0us/step


In [7]:
conv_base.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 77, 77, 64)   0           conv1_conv[0][0]                 
_________________________________________________________________________________________

## Feature extraction의 두 가지 방법
1. **빠른 추출방식**
    - 예측하려는 새로운 데이터를 위의 `conv_base`에 입력하여 나온 출력값을 numpy 배열로 저장하고 이를 분류 모델의 입력값으로 사용. Convolution operation을 하지 않아도 되기 때문에 빠르게 학습. 하지만 data augmentation 방법을 사용할 수 없음.

2. **받아온 특성 Layer를 이용해 새로운 모델 구현하는 방식**
    - 위의 `conv_base` 이후에 새로운 layer를 쌓아 확장한 뒤 전체 모델을 다시 학습. 모든 데이터가 convolution layer들을 통과해야 하기 때문에 학습이 느림. 단 conv_base의 가중치는 업데이트 되지 않도록 한다. data augmentation 방법을 사용할 수 있음.

### 빠른 특성 추출 방식


- `conv_base`의 predict 메소드로 입력 이미지의 feature를 추출 

In [12]:
# 하이퍼파라미터
LEARNING_RATE = 0.001
N_EPOCHS = 30
N_BATCHS = 100

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16, ResNet50V2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np

np.random.seed(1)
tf.random.set_seed(1)

In [13]:
def next_featuremap(image_directory, sample_counts):
    """
    매개변수로 받은 디렉토리의 이미지들을 Conv_base(VGG16) 모델을 통과시켜서 Featuremap들을 추출해 반환하는 함수
    [매개변수]
        image_directory: 이미지 데이터들이 있는 디렉토리
        sample_counts: 특성을 추출할 이미지 개수
    [반환값]
        tuple: (featuremap들, label)
    """
    conv_base = VGG16(weights='imagenet',
                      include_top=False,
                      input_shape=(150, 150, 3),
                      )
    # 결과를 담을 ndarray
    # featuremap 저장, conv_base의 마지막 layer의 output의 shape에 맞춘다.
    return_features = np.zeros(shape=(sample_counts, 4, 4, 512))  # 위에서 150,150,3의 출력이 4,4,512였기 때문에
    return_labels = np.zeros(shape=(sample_counts, ))  # label들을 저장

    datagen = ImageDataGenerator(rescale=1./255)
    iterator = datagen.flow_from_directory(image_directory,
                                           target_size=(150, 150),
                                           batch_size=N_BATCHS,
                                           class_mode='binary',
                                           )
    
    i = 0  # 반복횟수를 저장할 변수
    for input_batch, label_batch in iterator:  # (image, label)*batch크기(100) 만큼 튜플로 반환
        # input_batch를 conv_base에 넣어서 feature map을 추출 - model.predict(): 모델의 레이어들을 통과해서 나온 출력결과를 반환
        fm = conv_base.predict(input_batch)

        return_features[i*N_BATCHS: (i+1)*N_BATCHS] = fm
        return_labels[i*N_BATCHS: (i+1)*N_BATCHS] = label_batch
        i += 1
        if i*N_BATCHS >= sample_counts:  # 결과를 저장할 배열의 시작 index가 sample_counts보다 크면 반복을 멈춘다.
            break
    
    return return_features, return_labels


In [14]:
def next_featuremap(image_dir, sample_counts):
    conv_base = ResNet50V2(weights='imagenet',
                           include_top=False,
                           input_shape=(150, 150, 3),
                           )
    return_features = np.zeros(shape=(sample_counts, 5, 5, 2048))
    return_labels = np.zeros(shape=(sample_counts, ))

    datagen = ImageDataGenerator(rescale=1/255.)
    iterator = datagen.flow_from_directory(image_dir,
                                           target_size=(150, 150),
                                           batch_size=N_BATCHS,
                                           class_mode='binary'
                                           )
    i = 0
    for input_batch, label_batch in iterator:
        fm = conv_base.predict(input_batch)

        return_features[i*N_BATCHS: (i+1)*N_BATCHS] = fm
        return_labels[i*N_BATCHS: (i+1)*N_BATCHS] = label_batch
        i += 1
        if i*N_BATCHS >= sample_counts:
            break
    return return_features, return_labels

In [15]:
train_dir = '/content/data/cats_and_dogs_small/train'
validation_dir = '/content/data/cats_and_dogs_small/validation'
test_dir = '/content/data/cats_and_dogs_small/test'

# Featuremap 추출
train_features, train_labels = next_featuremap(train_dir, 2000)
validation_features, validation_labels = next_featuremap(validation_dir, 1000)
test_featues, test_labels = next_featuremap(test_dir, 1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [16]:
train_features.shape

(2000, 5, 5, 2048)

In [17]:
train_labels.shape

(2000,)

In [21]:
def create_model():
    # 분류기 모델만 생성
    model = keras.Sequential()
    # model.add(layers.Input(shape=(4, 4, 512)))
    model.add(layers.Input(shape=(5, 5, 2048)))
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

In [22]:
model = create_model()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss='binary_crossentropy',
              metrics=['accuracy']
              )
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               524544    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 524,801
Trainable params: 524,801
Non-trainable params: 0
_________________________________________________________________


In [23]:
# N_EPOCHS = 100
history = model.fit(train_features, train_labels,
                    epochs=N_EPOCHS,
                    batch_size=N_BATCHS,
                    validation_data=(validation_features,validation_labels),
                    )

Epoch 1/30
20/20 [==============================] - 1s 29ms/step - loss: 0.4370 - accuracy: 0.8397 - val_loss: 0.1551 - val_accuracy: 0.9470
Epoch 2/30
20/20 [==============================] - 0s 17ms/step - loss: 0.1555 - accuracy: 0.9579 - val_loss: 0.1127 - val_accuracy: 0.9550
Epoch 3/30
20/20 [==============================] - 0s 17ms/step - loss: 0.0646 - accuracy: 0.9773 - val_loss: 0.1037 - val_accuracy: 0.9590
Epoch 4/30
20/20 [==============================] - 0s 17ms/step - loss: 0.0304 - accuracy: 0.9934 - val_loss: 0.1112 - val_accuracy: 0.9580
Epoch 5/30
20/20 [==============================] - 0s 17ms/step - loss: 0.0186 - accuracy: 0.9983 - val_loss: 0.1131 - val_accuracy: 0.9600
Epoch 6/30
20/20 [==============================] - 0s 18ms/step - loss: 0.0149 - accuracy: 0.9971 - val_loss: 0.1229 - val_accuracy: 0.9600
Epoch 7/30
20/20 [==============================] - 0s 17ms/step - loss: 0.0070 - accuracy: 0.9998 - val_loss: 0.1167 - val_accuracy: 0.9620
Epoch 8/30
20

In [27]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# 한 개 이미지 추론
def predict_cat_dog(path, model, mode=False):
    class_name = ['cat', 'dog']
    img = load_img(path, target_size=(150, 150, 3))
    sample = img_to_array(img)[np.newaxis, ...]
    sample = sample/255.
    if mode:  # conv_base를 거치도록
        cb = ResNet50V2(weights='imagenet',
                        include_top=False,
                        input_shape=(150, 150, 3),
                        )
        sample = cb.predict(sample)

    pred = model.predict(sample)
    pred_class = np.where(pred < 0.5, 0, 1)
    pred_class_name = class_name[pred_class[0,0]]

    return pred, pred_class, pred_class_name


In [28]:
predict_cat_dog('/content/dog.jpg', model, mode=True)

(array([[0.9999888]], dtype=float32), array([[1]]), 'dog')

### Pretrained Network를 이용해 새로운 모델 구현하는 방식

- Conv_base의 feature extraction 부분에 fully connected layer를 추가하여 모형 생성 
- Conv_base에서 가져온 부분은 학습을 하지 않고 weight를 고정
    -  **Layer.trainable=False**

In [ ]:
LEARNING_RATE = 0.001
N_EPOCHS = 20
N_BATCHS = 100
IMAGE_SIZE = 150

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def create_model():
    conv_base = VGG16(weights='imagenet',
                      include_top=False,
                      input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)
                      )
    conv_base.trainable = False  # 학습 시 weight 최적화(update)를 하지 않도록 설정 -> 모델 컴파일 전에 실행해야 한다.

    model = keras.Sequential()
    model.add(conv_base)
    model.add(layers.GlobalAveragePooling2D())  # 4*4*512이므로 Flatten 대신 사용 -> 결과: 1*1*512
    model.add(layers.Dense(256, activation='relu'))

    # 출력
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

In [ ]:
model = create_model()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss='binary_crossentropy',
              metrics=['accuracy']
              )
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 14,846,273
Trainable params: 131,585
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
train_iterator, validation_iterator, test_iterator = get_generators()

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
history = model.fit(train_iterator,
                    epochs=N_EPOCHS,
                    steps_per_epoch=len(train_iterator),
                    validation_data=validation_iterator,
                    validation_steps=len(validation_iterator)
                    )

Epoch 1/20
20/20 [==============================] - 59s 981ms/step - loss: 0.6436 - accuracy: 0.6259 - val_loss: 0.4333 - val_accuracy: 0.8380
Epoch 2/20
20/20 [==============================] - 19s 969ms/step - loss: 0.4553 - accuracy: 0.8163 - val_loss: 0.3522 - val_accuracy: 0.8570
Epoch 3/20
20/20 [==============================] - 19s 960ms/step - loss: 0.3939 - accuracy: 0.8356 - val_loss: 0.3247 - val_accuracy: 0.8770
Epoch 4/20
20/20 [==============================] - 19s 962ms/step - loss: 0.3646 - accuracy: 0.8360 - val_loss: 0.3495 - val_accuracy: 0.8420
Epoch 5/20
20/20 [==============================] - 19s 963ms/step - loss: 0.3655 - accuracy: 0.8336 - val_loss: 0.2885 - val_accuracy: 0.8860
Epoch 6/20
20/20 [==============================] - 19s 967ms/step - loss: 0.3253 - accuracy: 0.8611 - val_loss: 0.2792 - val_accuracy: 0.8840
Epoch 7/20
20/20 [==============================] - 19s 966ms/step - loss: 0.3317 - accuracy: 0.8431 - val_loss: 0.2703 - val_accuracy: 0.8910

## 미세조정(Fine-tuning)
- Pretrained 모델을 내가 학습시켜야 하는 데이터셋(Custom Dataset)에 재학습시키는 것을 fine tunning 이라고 한다.
- 주어진 문제에 더 적합하도록 모델의 가중치들을 조정.

### Fine tuning 전략
![image-2.png](attachment:image-2.png)

- **세 전략 모두 classifier layer들은 train한다.**

1. <span style="font-size:1.2em;font-weight:bold">전체 모델을 전부 학습시킨다.(1번)</span>
    - Pretrained 모델의 weight는 Feature extraction 의 초기 weight 역할을 한다.
    - **Train dataset의 양이 많고** Pretrained 모델이 학습했던 dataset과 Custom dataset의 class간의 유사성이 **낮은 경우** 적용.
    - 학습에 시간이 많이 걸린다.
2. <span style="font-size:1.2em;font-weight:bold">Pretrained 모델 Bottom layer들(Input과 가까운 Layer들)은 고정시키고 Top layer의 일부를 재학습시킨다.(2번)</span>
    - **Train dataset의 양이 많고** Pretrained 모델이 학습했던 dataset과 Custom dataset의 class간의 유사성이 **높은 경우** 적용.
    - **Train dataset의 양이 적고** Pretained 모델이 학습했던 dataset과 custom dataset의 class간의 유사성이 **낮은 경우** 적용
3. <span style="font-size:1.2em;font-weight:bold">Pretrained 모델 전체를 고정시키고 classifier layer들만 학습시킨다.(3번)</span>
    - **Train dataset의 양이 적고** Pretrained 모델이 학습했던 dataset과 Custom dataset의 class간의 유사성이 **높은 경우** 적용.
  
  
> custom dataset: 내가 학습시키고자 하는 dataset 

In [7]:
cb = VGG16(include_top=False,
           weights='imagenet',
           input_shape=(150, 150, 3)
           )
cb.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
______________________________________________________________

In [9]:
# network(모델)을 구성하는 layer들을 추출

layers = cb.layers
type(layers), len(layers)  # model을 구성하는 layer들을 추출해서 list에 묶어서 반환

(list, 19)

In [11]:
layers[2].trainable = False

In [12]:
# layer의 이름을 조회
layers[2].name

'block1_conv2'

In [16]:
# 모델.get_layer('layer이름'): 지정한 이름의 layer를 반환
l = cb.get_layer('block1_conv2')
l

In [17]:
# layer의 가중치(weights)를 조회 - layer객체.weights
l_w = l.weights
type(l_w), len(l_w)  # [weight, bias]의 형태로 반환

(list, 2)

In [21]:
np.shape(l_w[0]), np.shape(l_w[1])

(TensorShape([3, 3, 64, 64]), TensorShape([64]))

In [22]:
del layers

### Pretrained 모델 Bottom layer들(Input과 가까운 Layer들)은 고정시키고 Top layer의 일부를 재학습

- Conv_base에서 가장 Top부분에 있는 레이어에 대해 fine-tuning.
    - 앞의 layer들은 비교적 일반적이고 재사용 가능한 feature를 학습
    - 너무 많은 parameter를 학습시키면 overfitting의 위험이 있음 (특히 새로운 데이터의 수가 적을 때)

In [26]:
from tensorflow.keras import layers

def create_model():
    # VGG16: block5_conv2, block5_conv3 두 개의 convolution layer들을 fine tuning
    conv_base = VGG16(weights='imagenet',
                      include_top=False,
                      input_shape=(150, 150, 3)
                      )
    # trainable 설정
    for layer in conv_base.layers:
        is_trainable = False
        if layer.name == 'block5_conv2' or layer.name == 'block5_conv3':
            is_trainable = True
        layer.trainable = is_trainable
    
    model = keras.Sequential()
    model.add(conv_base)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

In [27]:
model = create_model()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss='binary_crossentropy',
              metrics=['accuracy']
              )
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 14,846,273
Trainable params: 4,851,201
Non-trainable params: 9,995,072
_________________________________________________________________


In [29]:
train_iterator, validation_iterator, test_iterator = get_generators()

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [30]:
N_EPOCHS = 30
mc_callback = keras.callbacks.ModelCheckpoint('./models/cat_dog_model', 
                                              monitor='val_loss',
                                              save_best_only=True
                                              )
model.fit(train_iterator,
          epochs=N_EPOCHS,
          steps_per_epoch=len(train_iterator),
          validation_data=validation_iterator,
          validation_steps=len(validation_iterator),
          callbacks=[mc_callback]
          )

Epoch 1/30
20/20 [==============================] - 61s 986ms/step - loss: 1.0580 - accuracy: 0.5082 - val_loss: 0.4577 - val_accuracy: 0.7760
INFO:tensorflow:Assets written to: ./models/cat_dog_model/assets
Epoch 2/30
20/20 [==============================] - 20s 977ms/step - loss: 0.4480 - accuracy: 0.7864 - val_loss: 0.3245 - val_accuracy: 0.8520
INFO:tensorflow:Assets written to: ./models/cat_dog_model/assets
Epoch 3/30
20/20 [==============================] - 20s 983ms/step - loss: 0.3922 - accuracy: 0.8274 - val_loss: 0.2217 - val_accuracy: 0.9160
INFO:tensorflow:Assets written to: ./models/cat_dog_model/assets
Epoch 4/30
20/20 [==============================] - 20s 981ms/step - loss: 0.2862 - accuracy: 0.8774 - val_loss: 0.2574 - val_accuracy: 0.8880
INFO:tensorflow:Assets written to: ./models/cat_dog_model/assets
Epoch 5/30
20/20 [==============================] - 20s 983ms/step - loss: 0.2550 - accuracy: 0.8980 - val_loss: 0.2173 - val_accuracy: 0.9150
INFO:tensorflow:Assets wr

In [31]:
best_model = keras.models.load_model('./models/cat_dog_model')

In [32]:
# evaluation
best_model.evaluate(train_iterator)

20/20 [==============================] - 17s 813ms/step - loss: 0.0606 - accuracy: 0.9750


[0.06063142791390419, 0.9750000238418579]

In [33]:
best_model.evaluate(test_iterator)

10/10 [==============================] - 3s 306ms/step - loss: 0.2935 - accuracy: 0.9180


[0.29349854588508606, 0.9179999828338623]

In [35]:
predict_cat_dog('/content/dog.jpg', model, mode=False)

(array([[8.74577e-06]], dtype=float32), array([[0]]), 'cat')